<a href="https://colab.research.google.com/github/Maximilianwte/GenAI-Course/blob/main/BDD_GenAI_Train_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Maximilianwte/GenAI-Course.git
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
!pip install --upgrade diffusers -qq
!pip uninstall bitsandbytes -y -qq
!pip install bitsandbytes -qq
import os
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch
from torch import autocast
import numpy as np

fatal: destination path 'GenAI-Course' already exists and is not an empty directory.


In [ ]:
MODEL_NAME = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "/content/Test_Model"
!mkdir -p $OUTPUT_DIR

In [ ]:
!python3 train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --instance_data_dir='/content/GenAI-Course/Datasets/maskot-images' \
  --instance_prompt='a photo of xyzobject' \
  --seed=1 \
  --resolution=512 \
  --train_batch_size=2 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=800 \
  --save_min_steps=400 \
  --save_interval=200

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(OUTPUT_DIR, torch_dtype=torch.float16).to("cuda")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
g_cuda = torch.Generator(device='cuda')

In [ ]:
prompt = "a photo of xyzobject"
negative_prompt = ""

seed = np.random.randint(1, 1000)
g_cuda.manual_seed(seed)
guidance_scale = 7
with autocast("cuda"), torch.inference_mode():
    images = pipe(prompt, negative_prompt=negative_prompt, num_images_per_prompt=1, height=512, width=512, num_inference_steps=60, guidance_scale=guidance_scale, generator=g_cuda).images

images[0]